Importing required libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy

In [2]:
# Define constants
NUM_CLASSES = 10
BATCH_SIZE = 64
EPOCHS = 1
LEARNING_RATE = 0.001

In [3]:
# DenseNet was trained on ImageNet, so we use its mean/std for normalization.
# We also resize the CIFAR-10's 32x32 images to the 224x224 expected by the pre-trained model.
transform = transforms.Compose([
    transforms.Resize(224), # Resize images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [4]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


Loading data

In [5]:
# Load CIFAR-10 data
train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

100%|███████████████████████████████████████████████████████████████████████████████| 170M/170M [00:03<00:00, 43.5MB/s]


In [6]:
#reducing the number of images for faster training
from torch.utils.data import Subset
import random

SUBSET_SIZE = 5000  # Using only 5,000 images instead of 50,000

# Get the indices for the subset
indices = random.sample(range(len(train_dataset)), SUBSET_SIZE)

# Create the new reduced training dataset
train_subset = Subset(train_dataset, indices)

In [7]:
TEST_SUBSET_SIZE = 1000  # Example: Use 1,000 images instead of 10,000

# Get the indices for the subset
indices = random.sample(range(len(test_dataset)), TEST_SUBSET_SIZE)

# Create the new reduced test dataset
test_subset = Subset(test_dataset, indices)

In [8]:
# Create DataLoaders
train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_subset, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
print(f"Number of training batches: {len(train_loader)}")
print(f"Number of testing batches: {len(test_loader)}")

Number of training batches: 79
Number of testing batches: 16


Build the DenseNet121 Model (Transfer Learning)

In [10]:
def setup_densenet_model(num_classes):
  # Load the pre-trained DenseNet121 model
  model = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)

  # Freeze all the parameters (feature extractor layers)
  for param in model.parameters():
    param.requires_grad = False

  # Replace the final classification layer (classifier)
  # The original classifier has a linear layer whose input is num_features
  num_ftrs = model.classifier.in_features

  # Create a new classifier for 10 classes
  model.classifier = nn.Sequential(
      nn.Linear(num_ftrs, 512), # Optional intermediate layer
      nn.ReLU(),
      nn.Dropout(0.5),
      nn.Linear(512, num_classes) # Final layer for 10 classes
  )

  # Move model to the selected device
  model = model.to(device)

  return model

In [11]:
# Setup the model
model = setup_densenet_model(NUM_CLASSES)

In [12]:
# Define Loss function and Optimizer (only for the unfrozen layers)
criterion = nn.CrossEntropyLoss()

# We pass model.parameters() to the optimizer, but only the parameters
# of the new classifier head have requires_grad=True.
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

Training Loop

In [13]:
def train_model(model, criterion, optimizer, train_loader, epochs):
  history = {'train_loss': [], 'train_acc': []}

  for epoch in range(epochs):
    model.train() # Set the model to training mode
    running_loss = 0.0
    correct_preds = 0
    total_samples = 0

    for inputs, labels in train_loader:
      inputs, labels = inputs.to(device), labels.to(device)

      # Zero the parameter gradients
      optimizer.zero_grad()

      # Forward pass
      outputs = model(inputs)
      loss = criterion(outputs, labels)

      # Backward pass and optimize
      loss.backward()
      optimizer.step()

      # Statistics
      running_loss += loss.item() * inputs.size(0)
      _, predicted = torch.max(outputs.data, 1)
      total_samples += labels.size(0)
      correct_preds += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = correct_preds/total_samples

    history['train_loss'].append(epoch_loss)
    history['train_acc'].append(epoch_acc)

    print(f"Epoch {epoch+1}/{epochs} | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc*100:.2f}%")

  return history

In [14]:
#running the training
history = train_model(model, criterion, optimizer, train_loader, EPOCHS)

Epoch 1/1 | Loss: 1.3436 | Accuracy: 52.98%


Evaluation

In [15]:
def evaluate_model(model, test_loader):
  model.eval() # Set the model to evaluation model
  correct_preds = 0
  total_samples = 0

  with torch.no_grad(): # Disable gradient calculations during testing
    for inputs, labels in test_loader:
      inputs, labels = inputs.to(device), labels.to(device)

      outputs = model(inputs)
      _, predicted = torch.max(outputs.data, 1)
      total_samples += labels.size(0)
      correct_preds += (predicted == labels).sum().item()

  accuracy = correct_preds / total_samples
  print(f"\nFinal Test Accuracy: {accuracy*100:.2f}%")
  return accuracy

In [16]:
#run the evaluation
test_accuracy = evaluate_model(model, test_loader)


Final Test Accuracy: 73.50%


In [17]:
print(f"Test accuracy: {test_accuracy}")

Test accuracy: 0.735
